<a target="_blank" href="https://colab.research.google.com/github/marvlyngkhoi/efficeint_vlm_tools/blob/main/vllm_press.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Load images
image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")
# Initialize processor and model
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Base")
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Base",
    torch_dtype=torch.bfloat16,
    _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
     output_attentions=True,
     #return_dict_in_generate=True,
).to(DEVICE)



In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            #{"type": "image"},
            {"type": "text", "text": "What is the image about?"}
        ]
    },
]
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image2], return_tensors="pt")
inputs = inputs.to(DEVICE)

In [ ]:
from vllm_press import *

In [ ]:
# No compression
with torch.no_grad():
    generated_ids = model.generate(**inputs, max_new_tokens=100)
    
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)
print(generated_texts[0])


In [ ]:
press = KnormPress(0.5)
with torch.no_grad(), press(model):
    generated_ids = model.generate(**inputs, max_new_tokens=100)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)
print(generated_texts[0])